In [8]:
#Read in Redlist

import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
style.use('ggplot')

# Read in csv as dataframe
rl = pd.read_csv("redlistTable.csv")
# glance at first few rows
rl.head()

,Unnamed: 0,animalName,htmlPage,KINGDOM,PHYLUM,CLASS,ORDER,FAMILY,ECOLOGY,COUNTRIES,THREAT_PARAGRAPH,CONSERVATION_PARAGRAPH,POP_TREND,STATUS
0,0,Medionidus simpsonianus (Ochlockonee Moccasins...,12929.html,Animalia,Mollusca,Bivalvia,Unionoida,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n
1,0,Heteromys nelsoni (Nelson's Spiny Pocket Mouse),10009.html,Animalia,Chordata,Mammalia,Rodentia,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the s...,['\n This species occurs in a newly-named n...,Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n..."
2,0,Amauris comorana (Comoro Friar),1001.html,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n"
3,0,"Hippoglossus hippoglossus (Atlantic Halibut, H...",10097.html,Animalia,Chordata,Actinopterygii,Pleuronectiformes,Pleuronectidae,NaN,<div>Atlantic – northeast; Atlantic – northwes...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n
4,0,Hirasea acutissima,10171.html,Animalia,Mollusca,Gastropoda,Stylommatophora,Endodontidae,NaN,Japan (Ogasawara-shoto);,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n


In [24]:
#Read in ARKIVE
# Read in csv as dataframe
ar = pd.read_csv("arkive table analysis/output.csv", encoding="ISO-8859-1")
# glance at first few rows
ar.head()

,ÈÀName,Alternate Name,Kingdom,Phylum,Class,Order,Family,Genus,Size,Threats,...,x,y,z,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Alcatraz de Abbott,Piquero de Abbott,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula (1),Length: 79 cm (2),...,The majority of Abbott's booby nest sites occu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AbbotÛªs duiker,AbbottÛªs duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus (1),50 ÛÒ 60 kg (2),Like many other duikers Abbott\u2019s duiker i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cisticole des Aberdares,Aberdare cisticola,Animalia,Chordata,Aves,Passeriformes,Cisticolidae,Cisticola (1),Cisticola (1),[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adelaide pigmy blue-tongue skink,pygmy bluetongue,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua (1),Male snout-vent length: 3.8 - 10.6 cm (2)Femal...,...,The South Australia Department of Environment ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adriatic trout,softmouth trout,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo (1),80 - 425 g (4),...,There are currently no known conservation meas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# edit df to include only our 10 features of interest
rl = rl.ix[0:,['animalName','CLASS','ORDER','FAMILY','ECOLOGY','COUNTRIES','THREAT_PARAGRAPH','CONSERVATION_PARAGRAPH','POP_TREND','STATUS']]

In [11]:
# fix animalName to be just the animal's nickname/common name
animalNameVec = rl.ix[0:,['animalName']]
count = 0
total = 0
for index, row in rl.iterrows():
    nameString = row['animalName']
    #print(row['animalName'])
    #print(nameString)
    #print(nameString.find("/("))
    if nameString.find("/(")!=-1:
        count = count + 1
        total = 1 + total
    else:
        total = 1 + total

print(total)
print(count)
rl.head()
#df[df['animalName'].str.contains("()")]
#print(df['animalName'])
#df[df['animalName'].str.contains("(")]

#print(df.ix[15:15,["animalName"]])
#df.get_value(0,'animalName')

# set to updated name
#df.set_value('C', 'x', 10)

7915
0


,animalName,CLASS,ORDER,FAMILY,ECOLOGY,COUNTRIES,THREAT_PARAGRAPH,CONSERVATION_PARAGRAPH,POP_TREND,STATUS
0,Medionidus simpsonianus (Ochlockonee Moccasins...,Bivalvia,Unionoida,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n
1,Heteromys nelsoni (Nelson's Spiny Pocket Mouse),Mammalia,Rodentia,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the s...,['\n This species occurs in a newly-named n...,Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n..."
2,Amauris comorana (Comoro Friar),Insecta,Lepidoptera,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n"
3,"Hippoglossus hippoglossus (Atlantic Halibut, H...",Actinopterygii,Pleuronectiformes,Pleuronectidae,NaN,<div>Atlantic – northeast; Atlantic – northwes...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n
4,Hirasea acutissima,Gastropoda,Stylommatophora,Endodontidae,NaN,Japan (Ogasawara-shoto);,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n


In [15]:
# Add column to the table for country count
# author: @andrewedstrom
import operator, re

def buildCountryColumn(rl, indexString):
    newColumn = []
    for index, row in rl.iterrows():
        cellContents = row[indexString]
        if isinstance(cellContents, str):
            #cellContents = re.sub(r'\([^)]*\)', '', cellContents)
            cellContents = re.sub(r'<div>|</div>', '', cellContents)
            cellContents = cellContents.split(";")
            newColumn.append(len(cellContents) - 1)
            #newColumn.append(len(cellContents))
        else:
            newColumn.append(-1)
    return newColumn

df1 = rl[["COUNTRIES"]]
df1.head()
for column in df1:
    newColumn = buildCountryColumn(df1, column)
    rl['country_count'] = newColumn
rl.head()
    
        

,animalName,CLASS,ORDER,FAMILY,ECOLOGY,COUNTRIES,THREAT_PARAGRAPH,CONSERVATION_PARAGRAPH,POP_TREND,STATUS,country_count
0,Medionidus simpsonianus (Ochlockonee Moccasins...,Bivalvia,Unionoida,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,Heteromys nelsoni (Nelson's Spiny Pocket Mouse),Mammalia,Rodentia,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the s...,['\n This species occurs in a newly-named n...,Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n...",2
2,Amauris comorana (Comoro Friar),Insecta,Lepidoptera,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,"Hippoglossus hippoglossus (Atlantic Halibut, H...",Actinopterygii,Pleuronectiformes,Pleuronectidae,NaN,<div>Atlantic – northeast; Atlantic – northwes...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,Hirasea acutissima,Gastropoda,Stylommatophora,Endodontidae,NaN,Japan (Ogasawara-shoto);,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,1
